<a href="https://colab.research.google.com/github/LongNguyen1984/TimeSeriesWithPython/blob/main/Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Validation Set Approach

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

### Import data

In [4]:
df1 = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Auto.csv', na_values='?').dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 30.6+ KB


We begin by using the **sample()**
  function to split the set of observations into two halves, by selecting a random subset of 196 observations out of the original 392 observations. We refer to these observations as the training set.

We'll use the  **random_state**
  parameter in order to set a seed for **python**
 ’s random number generator, so that you'll obtain precisely the same results each time. It is generally a good idea to set a random seed when performing an analysis such as cross-validation that contains an element of randomness, so that the results obtained can be reproduced precisely at a later time.

In [12]:
train_df = df1.sample(196, random_state= 1)
test_df = df1[~df1.isin(train_df)].dropna(how='all')

X_train = train_df['horsepower'].values.reshape(-1,1) ### in the form array[[a], [b]]
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

In [13]:
X_test[0:10]

array([[165.],
       [150.],
       [150.],
       [215.],
       [170.],
       [ 95.],
       [ 87.],
       [ 90.],
       [ 95.],
       [ 90.]])

### Training and Predicting data
We then use  **LinearRegression()**
  to fit a linear regression to predict  **mpg**
  from  **horsepower**
  using only the observations corresponding to the training set.

In [23]:
model = skl_lm.LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
pred = model.predict(X_test)

In [25]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(pred, y_test)
print(MSE)

23.361902892587224


In [27]:
model.coef_, model.intercept_

(array([-0.15964606]), 40.3338030434159)

### Using polynomialFeatures() 
for polynomial and cubic regressions

In [28]:
from sklearn.preprocessing import PolynomialFeatures

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)

model.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

20.252690858345748


In [29]:
model.coef_, model.intercept_

(array([ 0.        , -0.51860376,  0.00141527]), 60.30223777216642)

In [30]:
# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)

model.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))

20.325609365972525


In [31]:
model.coef_, model.intercept_

(array([ 0.00000000e+00, -6.86758109e-01,  2.80449742e-03, -3.52379474e-06]),
 66.51996119784485)

In [35]:
train_df = df1.sample(196, random_state = 10)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

# Linear
model.fit(X_train, y_train)
print(mean_squared_error(y_test, model.predict(X_test)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)

model.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)

model.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))

26.997727618054043
20.734320863233357
20.94931591623871


## Leave-One-Out Cross-Validation (LOOCV)

In [42]:
model.fit(X_train, y_train)

from sklearn.model_selection import cross_val_score, LeaveOneOut


loo = LeaveOneOut()
X = df1['horsepower'].values.reshape(-1,1)
y = df1['mpg'].values.reshape(-1,1)
loo.get_n_splits(X)
### LOOCV equivalent KFOLD = N of dataset
from sklearn.model_selection import KFold

cv = KFold(n_splits=392, random_state = None, shuffle=False)
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)

print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


### Using LOOCV compare to KFold()

In [49]:
scores = []
for train_idx, test_idx in loo.split(X):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  model.fit(X_train, y_train)
  scores.append(mean_squared_error(y_test, model.predict(X_test)))

print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


Our cross-validation estimate for the test error is approximately 24.23. We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we use the for() function to initiate a for loop which iteratively fits polynomial regressions for polynomials of order i = 1 to i = 5 and computes the associated cross-validation error.

In [54]:
for i in range(1,6):
  poly = PolynomialFeatures(degree=i)
  X = df1['horsepower'].values.reshape(-1,1)
  y = df1['mpg'].values.reshape(-1,1)
  X_current = poly.fit_transform(X)
  #model.fit(X_current, y)
  scores = cross_val_score(model, X_current, y, scoring='neg_mean_squared_error', cv =cv, n_jobs=1)

  print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 24.231513517929226, STD: 36.797315036405344
Degree-2 polynomial MSE: 19.248213124489745, STD: 34.998446151782346
Degree-3 polynomial MSE: 19.33498406411498, STD: 35.76513567797254
Degree-4 polynomial MSE: 19.424430307079398, STD: 35.68335275228356
Degree-5 polynomial MSE: 19.033198669299832, STD: 35.31730233206761


### k-fold cross-validation

In [57]:
cv = KFold(n_splits=10, random_state=3, shuffle=False)

for i in range(1,11):
  poly = PolynomialFeatures(degree=i)
  X_current = poly.fit_transform(X)
  model = skl_lm.LinearRegression()
  #model = lm.fit(X_current, y)
  scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv = cv, n_jobs=1)
  print('Degree-'+str(i)+' polynomial MSE: '+ str(np.mean(np.abs(scores)))+ ", STD: "+ str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339864, STD: 14.510250711281133
Degree-2 polynomial MSE: 21.235840055801557, STD: 11.797327528897448
Degree-3 polynomial MSE: 21.336606183382052, STD: 11.844339714535948
Degree-4 polynomial MSE: 21.35388696930686, STD: 11.986332290218542
Degree-5 polynomial MSE: 20.905558736691994, STD: 12.18538879392164
Degree-6 polynomial MSE: 20.780544653505583, STD: 12.135161876519238
Degree-7 polynomial MSE: 20.95297059811304, STD: 12.059389652320029
Degree-8 polynomial MSE: 21.077108146456926, STD: 12.04455181676249
Degree-9 polynomial MSE: 21.035590598116677, STD: 11.948934876787705
Degree-10 polynomial MSE: 20.97800158252155, STD: 11.798777347111276


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


### An Application to Default Data

In [95]:
df2 = pd.read_csv('https://raw.githubusercontent.com/dsnair/ISLR/master/data/csv/Default.csv', na_values='?').dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [86]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, classification_report

for i in range(1,11):
  train_df2 = df2.sample(8000, random_state= i)
  test_df2 = df2[~df2.isin(train_df2)].dropna(how='all')

  # Fit a logistic regression to predict defualt using balance
 
  model = smf.glm('default~student' , data=train_df2, family = sm.families.Binomial())
  result = model.fit()
  predictions_nomial = ['Yes' if x <0.5 else 'No' for x in result.predict(test_df2)]
  print('------------------------------------')
  print('Random Seed = '+ str(i)+'')
  print('------------------------------------')
  print(confusion_matrix(test_df2['default'],
                         predictions_nomial)),
  print(classification_report(test_df2['default'],
                         predictions_nomial,
                         digits=3))
  print()

------------------------------------
Random Seed = 1
------------------------------------
[[1927    0]
 [  73    0]]
              precision    recall  f1-score   support

          No      0.964     1.000     0.981      1927
         Yes      0.000     0.000     0.000        73

    accuracy                          0.964      2000
   macro avg      0.482     0.500     0.491      2000
weighted avg      0.928     0.964     0.946      2000




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------
Random Seed = 2
------------------------------------
[[1932    0]
 [  68    0]]
              precision    recall  f1-score   support

          No      0.966     1.000     0.983      1932
         Yes      0.000     0.000     0.000        68

    accuracy                          0.966      2000
   macro avg      0.483     0.500     0.491      2000
weighted avg      0.933     0.966     0.949      2000




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------
Random Seed = 3
------------------------------------
[[1932    0]
 [  68    0]]
              precision    recall  f1-score   support

          No      0.966     1.000     0.983      1932
         Yes      0.000     0.000     0.000        68

    accuracy                          0.966      2000
   macro avg      0.483     0.500     0.491      2000
weighted avg      0.933     0.966     0.949      2000


------------------------------------
Random Seed = 4
------------------------------------
[[1936    0]
 [  64    0]]
              precision    recall  f1-score   support

          No      0.968     1.000     0.984      1936
         Yes      0.000     0.000     0.000        64

    accuracy                          0.968      2000
   macro avg      0.484     0.500     0.492      2000
weighted avg      0.937     0.968     0.952      2000




...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[1923    0]
 [  77    0]]
              precision    recall  f1-score   support

          No      0.962     1.000     0.980      1923
         Yes      0.000     0.000     0.000        77

    accuracy                          0.962      2000
   macro avg      0.481     0.500     0.490      2000
weighted avg      0.924     0.962     0.943      2000


------------------------------------
Random Seed = 10
------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[1932    0]
 [  68    0]]
              precision    recall  f1-score   support

          No      0.966     1.000     0.983      1932
         Yes      0.000     0.000     0.000        68

    accuracy                          0.966      2000
   macro avg      0.483     0.500     0.491      2000
weighted avg      0.933     0.966     0.949      2000




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Application 5-Fold estimation

In [109]:
from sklearn.model_selection import cross_val_score, KFold
#X = df2['balance'].values.reshape(-1,1)
#y = df2['default'].values
cv = KFold(n_splits= 5, random_state=3, shuffle=True)

for train_idx, test_idx in cv.split(df2):
    train = df2.loc[train_idx]
    test = df2.loc[test_idx]
    model = smf.glm('default~balance', data=train, family=sm.families.Binomial())
    predictions_nominal = [ "Yes" if x < 0.5 else "No" for x in result.predict(test)]
    print("----------------")
    print("Random Seed = " + str(i) + "")
    print("----------------")
    print(confusion_matrix(test["default"], 
                       predictions_nominal))
    print(classification_report(test["default"], 
                            predictions_nominal, 
                            digits = 3))
    print()

----------------
Random Seed = 10
----------------
[[1941    0]
 [  59    0]]
              precision    recall  f1-score   support

          No      0.971     1.000     0.985      1941
         Yes      0.000     0.000     0.000        59

    accuracy                          0.971      2000
   macro avg      0.485     0.500     0.493      2000
weighted avg      0.942     0.971     0.956      2000


----------------
Random Seed = 10
----------------
[[1924    0]
 [  76    0]]
              precision    recall  f1-score   support

          No      0.962     1.000     0.981      1924
         Yes      0.000     0.000     0.000        76

    accuracy                          0.962      2000
   macro avg      0.481     0.500     0.490      2000
weighted avg      0.925     0.962     0.943      2000


----------------
Random Seed = 10
----------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[1937    0]
 [  63    0]]
              precision    recall  f1-score   support

          No      0.969     1.000     0.984      1937
         Yes      0.000     0.000     0.000        63

    accuracy                          0.969      2000
   macro avg      0.484     0.500     0.492      2000
weighted avg      0.938     0.969     0.953      2000


----------------
Random Seed = 10
----------------
[[1933    0]
 [  67    0]]
              precision    recall  f1-score   support

          No      0.967     1.000     0.983      1933
         Yes      0.000     0.000     0.000        67

    accuracy                          0.967      2000
   macro avg      0.483     0.500     0.491      2000
weighted avg      0.934     0.967     0.950      2000


----------------
Random Seed = 10
----------------
[[1932    0]
 [  68    0]]
              precision    recall  f1-score   support

          No      0.966     1.000     0.983      1932
         Yes      0.000     0.000     0.000        6

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bootstrap

In [115]:
data = pd.read_csv('https://raw.githubusercontent.com/dsnair/ISLR/master/data/csv/Portfolio.csv')
data.info

<bound method DataFrame.info of            X         Y
0  -0.895251 -0.234924
1  -1.562454 -0.885176
2  -0.417090  0.271888
3   1.044356 -0.734198
4  -0.315568  0.841983
..       ...       ...
95  0.479091  1.454774
96 -0.535020 -0.399175
97 -0.773129 -0.957175
98  0.403634  1.396038
99 -0.588496 -0.497285

[100 rows x 2 columns]>

To illustrate the use of the bootstrap on this data, we must first create a function, **alpha()**, which takes as input the data and outputs the estimate for *α*
  (described in more detail on page 187).

In [111]:
def alpha(X,Y):
  return ((np.var(Y)-np.cov(X,Y))/(np.var(X)+np.var(Y) - 2*np.cov(X,Y)))

In [112]:
X = data.X[0:100]
y = data.Y[0:100]
print(alpha(X,y))

[[1.07270947 0.57665115]
 [0.57665115 0.06414064]]


In [120]:
dfsample = data.sample(frac=1, replace=True)
X = dfsample.X[0:100]
y = dfsample.Y[0:100]
print(alpha(X,y))

[[0.56372756 0.49864424]
 [0.49864424 0.58582664]]


In [121]:
def bstrap(df):
    tresult = 0
    for i in range(0,1000):
        dfsample = df.sample(frac=1, replace=True)
        X = dfsample.X[0:100]
        y = dfsample.Y[0:100]
        result = alpha(X,y)
        tresult += result
    fresult = tresult / 1000
    print(fresult)
bstrap(data)

[[ 1.0095905   0.57802585]
 [ 0.57802585 -0.12374097]]


###Estimating the Accuracy of a Linear Regression Model

In [126]:
from sklearn.utils import resample

auto_df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Auto.csv')

auto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


In [123]:
model = skl_lm.LinearRegression()
X = auto_df['horsepower'].values.reshape(-1,1)
y = auto_df['mpg']
model.fit(X,y)
print(model.coef_, model.intercept_)

[-0.15784473] 39.93586102117047


In [125]:
from sklearn.metrics import mean_squared_error

Xsamp, ysamp = resample(X, y, n_samples=1000)
model.fit(Xsamp, ysamp)
print('Intercept: ' + str(model.intercept_) + " Coef: " + str(model.coef_))

Intercept: 39.588405559254056 Coef: [-0.15602901]
